In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-06 21:53:47.031365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 21:53:47.124115: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 21:53:47.126923: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-06 21:53:47.126935: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [6]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
#     dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(lstm_layer1)
#     dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(lstm_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [7]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [8]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [ ]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

2023-07-06 21:53:49.952106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-06 21:53:49.952164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-06 21:53:49.952172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-06 21:53:49.952283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-06 21:53:49.952310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.116.04  Release Build  (dvs-builder@U16-I3-A16-3-3)  Thu Apr 27 18:11:06 UTC 2023
GCC version:  gcc 

Epoch 1/1000
289/289 [==============================] - 3s 4ms/step - loss: 691.4634 - val_loss: 849.1053
Epoch 2/1000
289/289 [==============================] - 1s 3ms/step - loss: 428.9437 - val_loss: 679.4831
Epoch 3/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5818 - val_loss: 671.8967
Epoch 4/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5261 - val_loss: 669.4630
Epoch 5/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5259 - val_loss: 672.7383
Epoch 6/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5331 - val_loss: 672.2908
Epoch 7/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5234 - val_loss: 667.7997
Epoch 8/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5292 - val_loss: 671.6800
Epoch 9/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5433 - val_loss: 671.0722
Epoch 10/1000
289/289 [=======================

289/289 [==============================] - 1s 2ms/step - loss: 411.5303 - val_loss: 671.3726
Epoch 78/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5838 - val_loss: 670.8093
Epoch 79/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5074 - val_loss: 670.6758
Epoch 80/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5101 - val_loss: 673.9629
Epoch 81/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5388 - val_loss: 669.9647
Epoch 82/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5453 - val_loss: 667.8781
Epoch 83/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5504 - val_loss: 671.0720
Epoch 84/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5455 - val_loss: 671.1788
Epoch 85/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5218 - val_loss: 669.3261
Epoch 86/1000
289/289 [============================

289/289 [==============================] - 1s 2ms/step - loss: 411.5492 - val_loss: 670.2642
Epoch 154/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5495 - val_loss: 671.0743
Epoch 155/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5438 - val_loss: 671.3777
Epoch 156/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5329 - val_loss: 671.3700
Epoch 157/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5119 - val_loss: 673.3416
Epoch 158/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5418 - val_loss: 672.3607
Epoch 159/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.5349 - val_loss: 672.9725
Epoch 160/1000
289/289 [==============================] - 2s 8ms/step - loss: 411.5642 - val_loss: 671.9335
Epoch 161/1000
289/289 [==============================] - 2s 9ms/step - loss: 411.5358 - val_loss: 670.4097
Epoch 162/1000
289/289 [===================

Epoch 229/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5269 - val_loss: 671.5192
Epoch 230/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5443 - val_loss: 669.0494
Epoch 231/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5166 - val_loss: 670.7081
Epoch 232/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.4613 - val_loss: 666.5757
Epoch 233/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5116 - val_loss: 671.4974
Epoch 234/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5695 - val_loss: 671.4645
Epoch 235/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5358 - val_loss: 672.6835
Epoch 236/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.5022 - val_loss: 670.4443
Epoch 237/1000
289/289 [==============================] - 1s 2ms/step - loss: 411.4574 - val_loss: 672.4921
Epoch 238/1000
289/289 [====

...

Epoch 217/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1114 - val_loss: 648.6529
Epoch 218/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0921 - val_loss: 646.0129
Epoch 219/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1280 - val_loss: 646.5862
Epoch 220/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0981 - val_loss: 643.4684
Epoch 221/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1105 - val_loss: 644.7087
Epoch 222/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1183 - val_loss: 648.2383
Epoch 223/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1029 - val_loss: 643.7049
Epoch 224/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0969 - val_loss: 645.4950
Epoch 225/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1073 - val_loss: 647.3925
Epoch 226/1000
320/320 [====

Epoch 293/1000
320/320 [==============================] - 2s 8ms/step - loss: 325.1254 - val_loss: 647.5383
Epoch 294/1000
320/320 [==============================] - 2s 8ms/step - loss: 325.0551 - val_loss: 643.2222
Epoch 295/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.1209 - val_loss: 643.6933
Epoch 296/1000
320/320 [==============================] - 2s 5ms/step - loss: 325.0733 - val_loss: 642.0524
Epoch 297/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0832 - val_loss: 649.4487
Epoch 298/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0907 - val_loss: 648.6370
Epoch 299/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0887 - val_loss: 649.4803
Epoch 300/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0999 - val_loss: 649.2362
Epoch 301/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1198 - val_loss: 647.3040
Epoch 302/1000
320/320 [====

Epoch 369/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0795 - val_loss: 643.3533
Epoch 370/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0207 - val_loss: 653.9026
Epoch 371/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1211 - val_loss: 647.4648
Epoch 372/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1484 - val_loss: 646.7005
Epoch 373/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0887 - val_loss: 649.5605
Epoch 374/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0999 - val_loss: 644.2115
Epoch 375/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0773 - val_loss: 650.4990
Epoch 376/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1265 - val_loss: 646.5914
Epoch 377/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0536 - val_loss: 643.0016
Epoch 378/1000
320/320 [====

Epoch 445/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1064 - val_loss: 649.6287
Epoch 446/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0823 - val_loss: 651.2388
Epoch 447/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1235 - val_loss: 647.8687
Epoch 448/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0960 - val_loss: 648.4977
Epoch 449/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0833 - val_loss: 643.6844
Epoch 450/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0832 - val_loss: 645.5978
Epoch 451/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1188 - val_loss: 649.2823
Epoch 452/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.1077 - val_loss: 646.3214
Epoch 453/1000
320/320 [==============================] - 1s 2ms/step - loss: 325.0697 - val_loss: 650.9755
Epoch 454/1000
320/320 [====

Epoch 521/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.1306 - val_loss: 646.0298
Epoch 522/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.0797 - val_loss: 650.6956
Epoch 523/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.0793 - val_loss: 650.8806
Epoch 524/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.1220 - val_loss: 647.7037
Epoch 525/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.0740 - val_loss: 649.4999
Epoch 526/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.0870 - val_loss: 650.9057
Epoch 527/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.0547 - val_loss: 653.7485
Epoch 528/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.1580 - val_loss: 648.5605
Epoch 529/1000
320/320 [==============================] - 2s 7ms/step - loss: 325.0264 - val_loss: 653.7524
Epoch 530/1000
320/320 [====

In [ ]:
mean_rmse, mean_mape = np.mean(rmse_list), np.mean(mape_list)          
mean_rmse, mean_mape

In [ ]:
rmse_list

In [ ]:
mape_list